In [41]:
%pip install spytial-diagramming
from spytial import *
from spytial.annotations import *

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [42]:
# Setup for performance metrics
import random
# Set this to True to run performance tests
RUN_PERF = False

perf_base = "spytial_perf"
def get_perf_path(structure, size):
    return perf_base + "_" + structure + "_" + f"{size}.json"
PI = 20
SIZES = [5, 10, 25, 50]


# Disjoint Sets

There are multiple possible visualizations here, depending on how much of the mechanization you might want.

- Just as sets of objects: Set the `VIEW` variable to `SET_ONLY` and run all.
- Just as a forest (as in CLRS): Set the `VIEW` variable to `FOREST_ONLY`
- Or as a combination of both: Set the `VIEW` variable to `FOREST+SET`

In [43]:
VIEW = "FOREST+SET"
#VIEW = "FOREST_ONLY"
#VIEW = "SET_ONLY"

In [44]:
@attribute(field="key")
@attribute(field="rank")
@hideAtom(selector="int + str")
@apply_if(
    VIEW == "FOREST_ONLY",
    orientation(selector='(parent - iden) & (DSUNode->DSUNode)', directions=['above']),
    orientation(selector='{ x, y : DSUNode | (x != y) and (@num:(x.rank) < @num:(y.rank)) }', directions=['above']),
)

@apply_if(
    VIEW == "SET_ONLY",
    group(selector='~parent', name='set'),
    hideField(field='parent'),
)
@apply_if(
    VIEW == "FOREST+SET",
    orientation(selector='(parent - iden) & (DSUNode->DSUNode)', directions=['above']),
    group(selector='^(~parent)', name='set'),
    orientation(selector='{ x, y : DSUNode | (x != y) and (@num:(x.rank) < @num:(y.rank)) }', directions=['above']),
)
class DSUNode:
    def __init__(self, key):
        self.key = key
        self.parent = self   # MAKE-SET
        self.rank = 0        # tree rank (upper bound on height)


class DisjointSet:
    def make_set(self, key):
        return DSUNode(key)

    # FIND-SET with path compression
    def find_set(self, x: DSUNode) -> DSUNode:
        if x.parent is not x:
            x.parent = self.find_set(x.parent)
        return x.parent

    # UNION by rank
    def union(self, x: DSUNode, y: DSUNode) -> DSUNode:
        return self._link(self.find_set(x), self.find_set(y))

    def _link(self, x_root: DSUNode, y_root: DSUNode) -> DSUNode:
        if x_root is y_root:
            return x_root
        if x_root.rank > y_root.rank:
            y_root.parent = x_root
            return x_root
        else:
            x_root.parent = y_root
            if x_root.rank == y_root.rank:
                y_root.rank += 1
            return y_root
        


CLRS: Fig 21.4 (Page 569)

![disjoint](./img/disjoint-set-forest.png)

In [45]:

# Create a DisjointSet instance and some DSU nodes, perform unions, and render a diagram.
ds = DisjointSet()

# MAKE-SET for elements
f = ds.make_set("f")
d = ds.make_set("d")
g = ds.make_set("c")

b = ds.make_set("b")
h = ds.make_set("h")
c = ds.make_set("c")
e = ds.make_set("e")  # will remain a separate set


ds.union(h, b)
ds.union(c, h)
ds.union(c,e)

ds.union(d,g)
ds.union(f, d)


to_diagram = hideAtom(selector="list")([f,d,g,b,h,c,e])
diagram(to_diagram)

ds.union(f, c)  
diagram(to_diagram)



## Performance

In [46]:
if RUN_PERF:
    STRUCTURE = "disjoint_set"
    for size in SIZES:
        # Create disjoint set with size elements
        ds = DisjointSet()
        elements = [ds.make_set(i) for i in range(size)]
        
        # Perform some random unions
        for _ in range(size // 2):
            i, j = random.sample(range(size), 2)
            ds.union(elements[i], elements[j])
        
        to_diagram = hideAtom(selector="list")(elements)
        diagram(to_diagram, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI, timeout=500, headless=True)
else:
    print("Performance testing skipped. Set RUN_PERF = True to enable.")


Performance testing skipped. Set RUN_PERF = True to enable.
